In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 

In [50]:
df = pd.read_csv("vacancies.csv") 

In [52]:
# Создаем целевую переменную
df['salary'] = np.where(
    df['russian_salary_from'].notna() & df['russian_salary_to'].notna(),
    (df['russian_salary_from'] + df['russian_salary_to']) / 2,
    np.where(
        df['russian_salary_from'].notna(),
        df['russian_salary_from'],
        df['russian_salary_to']
    )
)

df = df.dropna(subset=['salary'])

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1673 entries, 4 to 5575
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1673 non-null   int64  
 1   title                1673 non-null   object 
 2   company_name         1673 non-null   object 
 3   currency             1673 non-null   object 
 4   experience           1673 non-null   object 
 5   type_of_employment   1673 non-null   object 
 6   work_format          1673 non-null   object 
 7   description          1673 non-null   object 
 8   skills               1673 non-null   object 
 9   address              1673 non-null   object 
 10  published_at         1673 non-null   object 
 11  archived             1673 non-null   bool   
 12  url                  1673 non-null   object 
 13  min_experience       1673 non-null   int64  
 14  max_experience       1643 non-null   float64
 15  salary_to            868 non-null    float6

In [56]:
df = df.drop(columns=[
    'id', 'company_name', 'currency', 'published_at', 
    'archived', 'url', 'salary_to', 'salary_from', 'experience', 'russian_salary_to', 'russian_salary_from', 
    'description', 'title'
])

In [58]:
df.columns

Index(['type_of_employment', 'work_format', 'skills', 'address',
       'min_experience', 'max_experience', 'salary'],
      dtype='object')

In [60]:
print(df[['min_experience', 'max_experience', 'salary']].corr())

                min_experience  max_experience    salary
min_experience        1.000000        0.971802  0.550029
max_experience        0.971802        1.000000  0.522721
salary                0.550029        0.522721  1.000000


Можно оставить только min_experience, так как кореляция между этими признаками очень высокая

In [63]:
df = df.drop(columns=['max_experience'])

In [65]:
df['type_of_employment'].value_counts()

type_of_employment
Полная занятость       1623
Частичная занятость      35
Проектная работа          9
Стажировка                6
Name: count, dtype: int64

In [67]:
df['type_of_employment']=df['type_of_employment'].replace('Частичная занятость', 'Другое')
df['type_of_employment']=df['type_of_employment'].replace('Проектная работа', 'Другое')
df['type_of_employment']=df['type_of_employment'].replace('Стажировка', 'Другое')
df['type_of_employment'].value_counts()

type_of_employment
Полная занятость    1623
Другое                50
Name: count, dtype: int64

In [69]:
df['work_format'].value_counts()

work_format
Полный день         1152
Удаленная работа     504
Гибкий график          8
Сменный график         8
Вахтовый метод         1
Name: count, dtype: int64

In [71]:
df['work_format']=df['work_format'].replace('Сменный график', 'Гибкий график')
df = df[df['work_format'] != 'Вахтовый метод']
df['work_format'].value_counts()

work_format
Полный день         1152
Удаленная работа     504
Гибкий график         16
Name: count, dtype: int64

In [73]:
df = pd.get_dummies(df, columns=['type_of_employment', 'work_format'], dtype=int)

In [75]:
df

,skills,address,min_experience,salary,type_of_employment_Другое,type_of_employment_Полная занятость,work_format_Гибкий график,work_format_Полный день,work_format_Удаленная работа
4,"{SQL,Clickhouse,BigQuery,Databases,""Моделирова...",Москва,3,428677.92,0,1,0,0,1
18,"{SQL,Clickhouse,BigQuery,Databases,""Моделирова...",Москва,3,375645.60,0,1,0,0,1
24,"{SQL,Clickhouse,BigQuery,Databases,""Моделирова...",Москва,3,486129.60,0,1,0,0,1
25,"{Python,pandas,Regex,""Natural Language Process...",Санкт-Петербург,1,132580.80,1,0,0,0,1
27,"{Python,SQL,""Анализ данных"",Amplitude,""Английс...",Санкт-Петербург,1,70709.76,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
5571,"{""Работа с большим объемом информации"",Бюджети...",Краснодар,1,80000.00,0,1,0,0,1
5572,"{Ответственность,Внимательность,Коммуникабельн...",Новокузнецк,0,60000.00,0,1,0,1,0
5573,{},Москва,3,200000.00,0,1,0,1,0
5574,{},Тимашевск,1,55000.00,0,1,1,0,0


In [77]:
import re
from collections import Counter

def clean_skills(skill_str):
    skill_str = skill_str.strip('{}')
    skills = re.findall(r'(?:[^,"]|"(?:\\.|[^"])*")+', skill_str)
    skills = [s.strip().strip('"') for s in skills if s.strip()]
    return skills

df['skills_cleaned'] = df['skills'].apply(clean_skills)

selected_skills = [
    'SQL',
    'MS Excel',
    'Системный анализ',
    'Python',
    'Бизнес-анализ',
    'UML',
    'Работа с большим объемом информации',
    'Power BI',
    'Разработка технических заданий',
    'MS SQL',
    'REST',
    'Постановка задач разработчикам',
    'MS PowerPoint',
    'PostgreSQL',
    'API',
    'SOAP',
    'Atlassian Jira',
    'XML',
    'Финансовый анализ',
    'Tableau',
    'DWH',
    'Формирование аналитической отчетности',
    'Atlassian Confluence',
    'JSON',
    'Английский язык',
    'REST API',
    'Hadoop',
    'Визуализация данных',
    'ETL',
    'Clickhouse',
    'Математическая статистика',
    'Big Data',
    'Power Query',
    'ERP-системы на базе 1С',
    'Прогнозирование',
    '1С: Предприятие 8',
    'Экономический анализ',
    'Системное мышление',
    'Финансовая отчетность',
    'Системная интеграция',
    'Git',
    'MS Visio',
    'Greenplum',
    'JSON API',
    '1С: Бухгалтерия',
    'ORACLE',
    'Автоматизация процессов',
    'A/B тесты',
    'Postman',
    '1С: Документооборот',
    'Apache Kafka',
    'Scrum',
    'Оптимизация численности персонала',
    'Agile',
    'Бюджетирование',
    'Разработка бизнес-требований',
    'Статистический анализ',
    'Use case',
    'Аналитика продаж',
    '1C: ERP',
    'Финансовое планирование',
    'Разработка функциональных требований',
    'Управление проектами',
    '1С: Зарплата и управление персоналом',
    'СУБД',
    'Swagger',
    'Финансовое моделирование',
    'Kafka'
]

for skill in selected_skills:
    df[f'skill_{skill}'] = df['skills_cleaned'].apply(lambda x: 1 if skill in x else 0)

df = df.drop(columns=['skills', 'skills_cleaned'])

In [79]:
top_address = df['address'].value_counts().head(6).index
df['address_group'] = df['address'].apply(lambda x: x if x in top_address else 'other')
df = pd.get_dummies(df, columns=['address_group'], prefix='address', dtype=int)
df = df.drop(columns=['address'])

In [81]:
df.columns

Index(['min_experience', 'salary', 'type_of_employment_Другое',
       'type_of_employment_Полная занятость', 'work_format_Гибкий график',
       'work_format_Полный день', 'work_format_Удаленная работа', 'skill_SQL',
       'skill_MS Excel', 'skill_Системный анализ', 'skill_Python',
       'skill_Бизнес-анализ', 'skill_UML',
       'skill_Работа с большим объемом информации', 'skill_Power BI',
       'skill_Разработка технических заданий', 'skill_MS SQL', 'skill_REST',
       'skill_Постановка задач разработчикам', 'skill_MS PowerPoint',
       'skill_PostgreSQL', 'skill_API', 'skill_SOAP', 'skill_Atlassian Jira',
       'skill_XML', 'skill_Финансовый анализ', 'skill_Tableau', 'skill_DWH',
       'skill_Формирование аналитической отчетности',
       'skill_Atlassian Confluence', 'skill_JSON', 'skill_Английский язык',
       'skill_REST API', 'skill_Hadoop', 'skill_Визуализация данных',
       'skill_ETL', 'skill_Clickhouse', 'skill_Математическая статистика',
       'skill_Big Data', 

In [83]:
# Референтые группы - type_of_employment_Полная занятость , work_format_Удаленная работа, address_Москва
X = df.drop(columns=['salary', 'type_of_employment_Полная занятость', 'work_format_Удаленная работа' ,
                     'address_Москва'])  # Предикторы
y = df['salary']  # Целевая переменная

X = sm.add_constant(X)

model1 = sm.OLS(y, X).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                 salary   R-squared:                       0.582
Model:                            OLS   Adj. R-squared:                  0.562
Method:                 Least Squares   F-statistic:                     28.49
Date:                Mon, 07 Apr 2025   Prob (F-statistic):          7.83e-246
Time:                        18:09:47   Log-Likelihood:                -20478.
No. Observations:                1672   AIC:                         4.111e+04
Df Residuals:                    1593   BIC:                         4.154e+04
Df Model:                          78                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

In [85]:
# Флаг для контроля остановки
all_significant = False

while not all_significant:
    model = sm.OLS(y, X).fit()
    p_values = model.pvalues.drop('const', errors='ignore')
    
    if (p_values <= 0.05).all():
        all_significant = True
    else:
        max_p_value = p_values.max()
        if max_p_value > 0.05:
            feature_to_drop = p_values.idxmax()
            print(f"Удаляем признак {feature_to_drop} (p-value = {max_p_value:.4f})")
            X = X.drop(columns=[feature_to_drop])
        else:
            all_significant = True

print(model.summary())

Удаляем признак skill_Постановка задач разработчикам (p-value = 0.9812)
Удаляем признак skill_Atlassian Confluence (p-value = 0.9279)
Удаляем признак skill_Atlassian Jira (p-value = 0.9421)
Удаляем признак skill_PostgreSQL (p-value = 0.9087)
Удаляем признак skill_MS PowerPoint (p-value = 0.8853)
Удаляем признак skill_1С: Бухгалтерия (p-value = 0.8687)
Удаляем признак skill_Разработка технических заданий (p-value = 0.8143)
Удаляем признак skill_Postman (p-value = 0.7912)
Удаляем признак skill_Swagger (p-value = 0.8191)
Удаляем признак skill_API (p-value = 0.7903)
Удаляем признак skill_Power Query (p-value = 0.7550)
Удаляем признак skill_1С: Документооборот (p-value = 0.7564)
Удаляем признак skill_XML (p-value = 0.7020)
Удаляем признак skill_Управление проектами (p-value = 0.6667)
Удаляем признак skill_Big Data (p-value = 0.6154)
Удаляем признак skill_СУБД (p-value = 0.6225)
Удаляем признак skill_Бюджетирование (p-value = 0.5922)
Удаляем признак skill_Английский язык (p-value = 0.5457)
У